## Processing Comments and Detecting the Sentiment behind the Comment

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import spacy
from spacy import displacy
from spacy.lang.en.stop_words import STOP_WORDS

In [2]:
data_yelp = pd.read_csv('yelp_labelled.txt', sep = '\t', header = None)

In [3]:
nlp = spacy.load('en_core_web_sm')

In [4]:
column_name = ['Reviews', 'Sentiment']
data_yelp.columns = column_name
data_yelp.head()

,Reviews,Sentiment
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [5]:
data_yelp.shape

(1000, 2)

In [6]:
data_amazon = pd.read_csv('amazon_cells_labelled.txt',sep = '\t', header = None )
data_amazon.columns = column_name
data_imdb = pd.read_csv('imdb_labelled.txt',sep = '\t', header = None )
data_imdb.columns = column_name
data = data_yelp.append([data_amazon, data_imdb], ignore_index = True)
data.shape

(2748, 2)

In [7]:
data.head()

,Reviews,Sentiment
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [8]:
data.Sentiment.value_counts()

1    1386
0    1362
Name: Sentiment, dtype: int64

In [9]:
data.isnull().sum()

Reviews      0
Sentiment    0
dtype: int64

In [10]:
import string
punct =  string.punctuation
punct

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [11]:
stopwords = list(STOP_WORDS)
print(stopwords)

['rather', 'just', 'these', 'give', 'made', 'due', 'down', 'should', 'part', 'were', 'someone', 'around', 'must', 'too', 'indeed', 'upon', 'otherwise', 'own', 'than', 'whereas', 'either', 'another', 'none', 'ourselves', 'top', 'please', 'beforehand', 'herein', 'with', 'move', 'that', 'during', 'less', '’m', 'through', 'afterwards', '’ll', '‘re', 'via', 'third', 'five', 'noone', 'could', 'when', 'did', 'whereupon', 'therein', 'somewhere', 'towards', 'one', 'seeming', 'had', 'once', 'namely', 'latter', 'call', 'hundred', 'really', 'thereupon', 'have', 'well', '‘ve', 'name', 'here', 'sometimes', 'anyway', 'those', 'against', 'would', 'done', 'mostly', 'hereby', 'am', 'anyhow', 'at', 'by', 'side', 'my', 'an', 'onto', 'else', 'anyone', 'some', 'always', 'thereafter', 'for', 'amount', 'full', 'take', 'itself', 'n‘t', 'hers', 'whatever', 'six', 'few', 'be', 'i', 'him', 'often', 'into', 'might', 'on', 'myself', 'such', 'until', 'becomes', 'yourself', "'re", 'though', 'whereby', 'out', 'besides

## Tokenization

In [12]:
def text_data_cleaning(sentence):
    doc = nlp(sentence) 
    
    tokens = []
    for token in doc:
        if token.lemma_ != "-PRON-":
            temp = token.lemma_.lower().strip()
        else:
            temp = token.lower_
        tokens.append(temp)
    cleaned_tokens = []
    for token in tokens:
        if token not in stopwords and token not in punct:
            cleaned_tokens.append(token)
    return cleaned_tokens

In [13]:
text_data_cleaning("Hello how are you. Like this video")

['hello', 'like', 'video']

In [14]:
from sklearn.svm import LinearSVC

In [15]:
tfidf = TfidfVectorizer(tokenizer = text_data_cleaning)
classifier = LinearSVC()

In [16]:
X = data['Reviews']
y = data['Sentiment']

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [18]:
X_train.shape, X_test.shape

((2198,), (550,))

## Creating Pipeline

In [19]:
model = Pipeline([('tfidf', tfidf), ('model', classifier)])

In [20]:
model.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function text_data_cleaning at 0x000002B3A956C438>,
                                 use_idf=True, vocabulary=None)),
                ('model',
                 LinearSVC(C=1.0, class_weight=None, dual=True,
                           fit_int

In [21]:
y_predict = model.predict(X_test)

## Classification Report

In [22]:
print(classification_report(y_test, y_predict))

              precision    recall  f1-score   support

           0       0.78      0.81      0.79       285
           1       0.79      0.75      0.77       265

    accuracy                           0.78       550
   macro avg       0.78      0.78      0.78       550
weighted avg       0.78      0.78      0.78       550



## Confusion Matrix

In [23]:
confusion_matrix(y_test, y_predict)

array([[231,  54],
       [ 67, 198]], dtype=int64)

## Accuracy Score

In [24]:
accuracy_score(y_test, y_predict)

0.78

## Detecting whether the Comment is Positive or Negative

In [25]:
comment = input("Paste your Comment:")
detect = model.predict([comment])
if detect == [1]:
    print("\n\n\n\nIt is a Positive Comment.")
elif detect == [0]:
    print("\n\n\n\nIt is a Negative Comment")

Paste your Comment:Wow, This is an amazing lesson




It is a Positive Comment.
